In [1]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers.pooling import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
import os
import numpy as np
from keras.applications.imagenet_utils import preprocess_input 
from keras.utils import np_utils
import random
import pprint
import cPickle
from scipy.misc import imread, imresize, imsave
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.engine.topology import Merge, merge

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        featurewise_center=True,
        rescale=1./255,
        dim_ordering='tf')

In [3]:
def rgb_to_yuv(inp):
    cnv = np.array([[0.299, 0.587, 0.114], [-0.14713, -0.28886, 0.436], [0.615, -0.51499, -0.10001]])
    return np.dot(inp, cnv.T)

random.seed(0)

X_train = []
Y_train = []

for directory in os.listdir('./data/'):
    for i,image in enumerate(random.sample(os.listdir('./data/' + directory),3)):
        img = load_img('./data/' + directory + '/' + image, target_size=(224,224))  # this is a PIL image
        x = img_to_array(img)
        x_yuv = rgb_to_yuv(x)
        Y_train.append(x_yuv[:,:,1:])
        X_train.append(x_yuv[:,:,:1])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [4]:
XX_train = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3] + 2))
XX_train[:,:,:,0:] = X_train
XX_train.shape

(33, 224, 224, 3)

In [5]:
def getModel():
    ''' 
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    for i in range(9):
        vgg_model.layers.pop()    
    
    bnl = BatchNormalization() (vgg_model.layers[-1].output)
    cnl = Convolution2D(256, 1, 1, border_mode='same') (bnl)
    usl = UpSampling2D() (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-5].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(128, 3, 3, border_mode='same') (ml)

    usl = UpSampling2D() (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-9].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(64, 3, 3, border_mode='same') (ml)

    usl = UpSampling2D() (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-12].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(3, 3, 3, border_mode='same') (ml)

    bnl = BatchNormalization() (vgg_model.layers[-14].output)
    ml = merge([cnl, bnl], mode='sum')
    cnl = Convolution2D(3, 3, 3, border_mode='same') (ml)

    cnl = Convolution2D(2, 3, 3, border_mode='same') (cnl)

    tl_model = Model( input=vgg_model.input, output=cnl )
    for i in range(1, 14):
        tl_model.layers[i].trainable = False

    return tl_model

In [6]:
##loss function

# def blur_uv_loss(rgb, inferred_rgb):
#     uv = rgb2uv(rgb)
#     uv_blur0 = rgb2uv(blur(rgb, 3))
#     uv_blur1 = rgb2uv(blur(rgb, 5))

#     inferred_uv = rgb2uv(inferred_rgb)
#     inferred_uv_blur0 = rgb2uv(blur(inferred_rgb, 3))
#     inferred_uv_blur1 = rgb2uv(blur(inferred_rgb, 5))

#     return  ( dist(inferred_uv, uv) +
#             dist(inferred_uv_blur0 , uv_blur0) +
#             dist(inferred_uv_blur1, uv_blur1) ) / 3

In [7]:
tl_model = getModel()
# pprint.PrettyPrinter(indent=2).pprint(tl_model.get_config())
tl_model.summary()

tl_model.compile(loss="mse", optimizer="adagrad", metrics=["acc"])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [8]:
train_datagen.fit(X_train)

In [ ]:
# history = tl_model.fit_generator(train_datagen.flow(X_train,Y_train,batch_size=88), samples_per_epoch=88, nb_epoch=50, validation_data=(X_valid,Y_valid), verbose=1)

history = tl_model.fit(XX_train,Y_train,batch_size=22, nb_epoch=20, verbose=1, shuffle=True)

# tl_model.save(filepath='./output/urban_2_images')
# cPickle.dump( history.history, open( "urbnan_2_images.p", "wb" ) )

Epoch 1/20
33/33 [==============================] - 83s - loss: 1936.3419 - acc: 0.4788    